<a href="https://colab.research.google.com/github/Sravani-05/Assignment04/blob/main/Keras_cv_data_aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras-cv --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.9/634.9 kB 9.4 MB/s eta 0:00:00


Creating a preprocessing pipeline

In [2]:
import keras_cv
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

augmenter = keras.Sequential(
  layers=[
      keras_cv.layers.RandomFlip(),
      keras_cv.layers.RandAugment(value_range=(0, 255)),
      keras_cv.layers.CutMix(),
      keras_cv.layers.MixUp()
    ]
)

def augment_data(images, labels):
  labels = tf.one_hot(labels, 3)
  inputs = {"images": images, "labels": labels}
  outputs = augmenter(inputs)
  return outputs['images'], outputs['labels']

You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.


Augment a tf.data.Dataset

In [3]:
dataset = tfds.load('rock_paper_scissors', as_supervised=True, split='train')
dataset = dataset.batch(8)
dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteHS3STW/rock_paper_scissors-train.tfrec…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteHS3STW/rock_paper_scissors-test.tfreco…

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


Create a model

In [4]:
densenet = keras_cv.models.DenseNet121(
  include_rescaling=True,
  include_top=True,
  classes=3
)
densenet.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Train the model with the prepared dataset

In [5]:
densenet.fit(dataset)

315/315 [==============================] - 3691s 12s/step - loss: 1.1592 - accuracy: 0.3397


Testing the model

Load the dataset and Evaluate the model

In [6]:
test_dataset = tfds.load('rock_paper_scissors', as_supervised=True, split='test')
test_dataset = test_dataset.batch(64)
test_dataset = test_dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)

loss, accuracy = densenet.evaluate(test_dataset)

print('Test loss:', loss)
print('Test accuracy:', accuracy)


6/6 [==============================] - 139s 21s/step - loss: 2.8671 - accuracy: 0.3387
Test loss: 2.8670594692230225
Test accuracy: 0.33870968222618103


Create image using model 


In [ ]:
from PIL import Image
from keras_cv.models import StableDiffusion

model = StableDiffusion(img_height=512, img_width=512, jit_compile=True)
img = model.text_to_image("image of a yellow bird")
Image.fromarray(img[0]).save("/content/yellow-bird.jpg")